# Logger

> A basic logging class

In [ ]:
# | default_exp Logger

In [ ]:
# | hide
from nbdev.showdoc import *
from fastcore.test import test_eq

In [ ]:
# | exporti
import aiohttp
import asyncio

from typing import Optional, List
from abc import ABC

import datetime as dt

from fastcore.basics import patch_to


In [ ]:
# | export


class Logger:
    """log class with user customizeable output method"""

    app_name: str
    logs: List[dict]

    output_fn: Optional[callable] = None     # function to call with write_logs method.

    def __init__(
        self,
        app_name: str,
        output_fn: Optional[
            callable
        ] = None,  # function to call with write_logs method.
    ):

        self.app_name = app_name
        self.output_fn = output_fn
        self.logs = []

    def _AddLog(self, message: str, type_str: str, debug=False):
        """internal method to append message to log"""

        new_row = {
            "date_time": dt.datetime.now(),
            "application": self.app_name,
            "type": type_str,
            "message": message,
        }

        if debug:
            print(new_row)

        self.logs.append(new_row)
 

## Logger logging methods

In [ ]:
#| export

@patch_to(Logger)
def log_info(self, message, debug=False):
    """log an informational message"""
    self._AddLog(message=message, type_str="Info", debug=debug)

@patch_to(Logger)
def log_error(self, message, debug=False):
    """log an error message"""
    self._AddLog(message=message, type_str="Error", debug=debug)

@patch_to(Logger)
def log_warning(self, message, debug=False):
    """log a warning message"""
    self._AddLog(message=message, type_str="Warning", debug=debug)

## Outputting Logs
During Logger instantiation, users can pass a function, `output_fn` which will be called with the `Logger.output_log` method

In [ ]:
#| export

@patch_to(Logger)
def output_log(self):
    """calls the user defined output function"""
    return self.output_fn(self.logs)

##### Sample implementation with a custom write_logs method

In [ ]:
import pandas as pd

def custom_write_logs_fn(logs):
    print("printing logs")
    return pd.DataFrame(logs)
    
logger = Logger(app_name="test", output_fn=custom_write_logs_fn)

logger.log_info("hello world")

log_df = logger.output_log()

log_df

printing logs


,date_time,application,type,message
0,2022-12-21 14:25:43.147021,test,Info,hello world


In [ ]:
#| hide
# test that custom_write_logs_fn is stored in the logger as output_fn.
test_eq(logger.output_fn, custom_write_logs_fn)


In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()
